#crawling


## 패키지 설치

In [ ]:
!pip install selenium
!apt-get update

!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver '/content/drive/MyDrive/Colab Notebooks'
!pip install chromedriver-autoinstaller

!pip install bitsandbytes

!pip install streamlit pyngrok

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

## 크롤링(베스트순 50개)

In [ ]:
# !apt-get install -y chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [ ]:
# !pip install selenium
# !pip install webdriver-manager
# !pip install openpyxl

In [ ]:
!python --version

import selenium
print(selenium.__version__)

Python 3.11.13
4.33.0


In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import sys
from selenium.webdriver.common.keys import Keys
import urllib.request
import os
from urllib.request import urlretrieve

import time
import pandas as pd
import chromedriver_autoinstaller

chrome_path = "/content/drive/MyDrive/Colab Notebooks/chromedriver"

sys.path.insert(0,chrome_path)
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')  # set path to chromedriver as per your configuration
chrome_options.add_argument('lang=ko_KR') # 한국어
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                            "AppleWebKit/537.36 (KHTML, like Gecko) "
                            "Chrome/114.0.0.0 Safari/537.36")

chromedriver_autoinstaller.install()  # set the target URL

In [ ]:
import re
import traceback
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# 크롬 옵션 설정
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('lang=ko_KR')
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                            "AppleWebKit/537.36 (KHTML, like Gecko) "
                            "Chrome/114.0.0.0 Safari/537.36")


class TodayHousePageNumberCrawler:
    def __init__(self, url):
        self.url = url
        self.driver = webdriver.Chrome(options=chrome_options)
        self.wait = WebDriverWait(self.driver, 10)
        self.seen_reviews = set()

    def collect_reviews(self, max_reviews=50):
        self.driver.get(self.url)
        collected_reviews = []

        page = 1
        while len(collected_reviews) < max_reviews:
            try:
                self.wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "p.production-review-item__description")))
                tags = self.driver.find_elements(By.CSS_SELECTOR, "p.production-review-item__description")

                for tag in tags:
                    if len(collected_reviews) >= max_reviews:
                        break
                    review_text = tag.text.strip()
                    if review_text and review_text not in self.seen_reviews:
                        collected_reviews.append(review_text)
                        self.seen_reviews.add(review_text)

                if len(collected_reviews) >= max_reviews:
                    break

                page += 1
                page_button_selector = f"button._3b4ci[aria-label='{page}페이지']"
                next_page_btn = self.driver.find_element(By.CSS_SELECTOR, page_button_selector)
                self.driver.execute_script("arguments[0].click();", next_page_btn)

                self.wait.until(EC.staleness_of(tags[0]))
                self.wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "p.production-review-item__description")))

            except Exception as e:
                print(f"[INFO] {page}페이지 없음 또는 이동 실패: {e}")
                break

        self.driver.quit()
        return collected_reviews


def crawl_todayhouse_50_reviews(url):
    match = re.search(r"/productions/(\d+)", url)
    if not match:
        raise ValueError("올바른 오늘의집 제품 URL이 아닙니다.")

    crawler = TodayHousePageNumberCrawler(url)
    reviews = crawler.collect_reviews(max_reviews=50)

    cleaned_reviews = [r.replace("\n", " ").strip() for r in reviews]
    df = pd.DataFrame(cleaned_reviews, columns=["Review"])
    return df


# 실행부
url = input("오늘의집 제품 URL을 입력하세요: ").strip()
df = crawl_todayhouse_50_reviews(url)

# 전체 리뷰 출력
print(f"\n[총 {len(df)}개 리뷰 전체 출력]\n")
for i, review in enumerate(df["Review"], 1):
    print(f"{i:>2}. {review}")


오늘의집 제품 URL을 입력하세요: https://ohou.se/productions/1972953/selling
[INFO] 2페이지 없음 또는 이동 실패: Message: no such element: Unable to locate element: {"method":"css selector","selector":"button._3b4ci[aria-label='2페이지']"}
  (Session info: chrome=137.0.7151.119); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x59e99394c45a <unknown>
#1 0x59e9933f1760 <unknown>
#2 0x59e993443127 <unknown>
#3 0x59e993443321 <unknown>
#4 0x59e993491e94 <unknown>
#5 0x59e993468e7d <unknown>
#6 0x59e99348f30b <unknown>
#7 0x59e993468c23 <unknown>
#8 0x59e9934354a5 <unknown>
#9 0x59e993436111 <unknown>
#10 0x59e993910f1b <unknown>
#11 0x59e993914e19 <unknown>
#12 0x59e9938f7ac9 <unknown>
#13 0x59e9939159c8 <unknown>
#14 0x59e9938dc34f <unknown>
#15 0x59e993939a28 <unknown>
#16 0x59e993939c06 <unknown>
#17 0x59e99394b336 <unknown>
#18 0x7a69d254eac3 <unknown>


[총 5개 리뷰 전체 출력]

 1. 장마기간을 준비하면서 구입했어요 바빠서 아직 

# 요약


## Gemma

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGrained).
The token `kubig-llm` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticat

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

model_name = "google/gemma-7b-it"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
    return_full_text=False
)

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

...

model-00001-of-00005.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
!pip install bitsandbytes

In [ ]:
import os
import math
import torch
import pandas as pd
import bitsandbytes as bnb
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

#4비트 양자화 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 4비트 양자화 활성화
    bnb_4bit_use_double_quant=True,  # double quantization 사용 여부
    bnb_4bit_quant_type="nf4",  # 양자화 방식 (예: nf4)
    bnb_4bit_compute_dtype=torch.float16  # 연산 시 사용할 데이터 타입
)

pd.options.display.max_columns = 100

In [ ]:
#LoRA 설정
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

#LoRA 적용 및 모델 변환
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
model.half()                     # 모든 파라미터를 FP16으로

OutOfMemoryError: CUDA out of memory. Tried to allocate 3.75 GiB. GPU 0 has a total capacity of 39.56 GiB of which 654.88 MiB is free. Process 18386 has 38.91 GiB memory in use. Of the allocated memory 38.18 GiB is allocated by PyTorch, and 234.94 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

###토마토


In [ ]:
import pandas as pd

# 파일 경로
file_path = '/content/drive/MyDrive/KUBIG/25_1_conference/crawling_result/3109414_전체리뷰_전처리결과.txt'

# 줄 단위로 읽기 (공백 포함)
with open(file_path, 'r', encoding='utf-8') as f:
    lines = f.readlines()
# [전체 리뷰 - 50개] 같은 0번째 줄 제거
if lines and "[전체 리뷰" in lines[0]:
    lines = lines[1:]

# 줄바꿈 및 공백 제거
cleaned_reviews = [line.strip() for line in lines if line.strip()]
df_1 = pd.DataFrame(cleaned_reviews, columns=['review'])

# 결과 출력
print(df_1)


In [ ]:
import pandas as pd
import torch
import gc

# 텍스트 생성 함수
def generate_summary(prompt, max_new_tokens=300, temperature=0.7):
    result = pipe(prompt, max_new_tokens=max_new_tokens, do_sample=True, temperature=temperature)
    return result[0]["generated_text"].strip()

# 프롬프트 생성 함수 + 프롬프트 길이 제한
def build_prompt(reviews, sentiment, max_prompt_length=3500):
    # 최대 길이를 초과하지 않도록 리뷰 줄 수 조절
    joined = ""
    limited_reviews = []
    for r in reviews:
        if len(joined) + len(r) + 1 > max_prompt_length:
            break
        limited_reviews.append(r)
        joined += r + "\n"

    reviews_text = "\n".join(limited_reviews)
    if sentiment == "both":
        prompt = f"""
당신은 제품 리뷰를 분석하는 전문가입니다.

아래는 고객들이 작성한 실제 리뷰입니다.
중복된 표현은 묶어서 정리하고, 핵심적인 **긍정적인 점**과 **아쉬운 점**을 각각 간결하게 3개씩 요약해 주세요.

- 각 항목은 **한 문장**으로 작성해 주세요.
- **오타 없이** 맞춤법에 맞게 작성해 주세요.
- 불분명하거나 제품과 관련 없는 표현은 제거해 주세요.

[리뷰 목록]
{reviews_text}

[요약 결과]

긍정적인 점:
1.

2.

3.

아쉬운 점:
1.

2.

3.
"""
        print(f"[사용된 리뷰 수] {len(limited_reviews)}개")
        print(f"[프롬프트 길이] {len(prompt)}자")
        return prompt

# 전체 요약 실행
def summarize_all(reviews, sentiment):
    try:
        prompt = build_prompt(reviews, sentiment)
        summary = generate_summary(prompt)
        torch.cuda.empty_cache()
        gc.collect()
        return summary
    except Exception as e:
        print(f"[ERROR] 전체 요약 실패: {e}")
        return ""


# 실행
total_reviews = df_1["review"].dropna().tolist()
final_summary = summarize_all(total_reviews, "both")

print("[전체 리뷰 요약 결과]")
print(final_summary)


[사용된 리뷰 수] 24개
[프롬프트 길이] 3745자
[전체 리뷰 요약 결과]
```

**긍정적인 점:**
1. 두터른 수건으로 물기를 잘 흡수합니다.
2. 디자인이 호텔에서 봤던 그런 디자인이라 수건장에 넣어두니 수건 하나만으로도 뭔가 삶의질이 확 올라간 느낌이 드네요.
3. 편안한 촉감과 좋은 흡수력으로 사용 후에도 쾌적함을 유지할 수 있습니다.

**아쉬운 점:**
1. 세탁 후 변형이나 색 바램이 많이 발생합니다.
2. 먼지가 많이 나오는 경우가 많습니다.
3. 수건 재질이 약하여 잘게 뜨거워지거나 엉친다.


### 드라이거치대

In [ ]:
import pandas as pd

# 파일 경로
file_path = '/content/drive/MyDrive/KUBIG/25_1_conference/crawling_result/1737677_전체리뷰_전처리결과.txt'

# 줄 단위로 읽기 (공백 포함)
with open(file_path, 'r', encoding='utf-8') as f:
    lines = f.readlines()
# [전체 리뷰 - 50개] 같은 0번째 줄 제거
if lines and "[전체 리뷰" in lines[0]:
    lines = lines[1:]

# 줄바꿈 및 공백 제거
cleaned_reviews = [line.strip() for line in lines if line.strip()]
df_1 = pd.DataFrame(cleaned_reviews, columns=['review'])

# 결과 출력
print(df_1)


                                               review
0   화장대가 작은데 다이슨 슈퍼소닉 거치대와 구성품들을 보관하기 어려워서 측면에 달기 ...
1   신혼 첫집부터 벌써 세번째 집인데 항상 이거 재구매해서 사용중입니다 무타공 마켓은 ...
2   화장실 타일 벽에 아무리 해도 본드가 안붙어서 위치가 좀 애매하지만 잘 쓰고 있습니...
3   화장실에 드라이거치대 두고 쓰는중입니다 아직까진 만족스럽습니다 단점이라면고무패킹이 ...
4   타일에 부착하기 좋은 거치대 드라이기를 방에서 사용하다보니 벽지에 부착하게 되었는데...
5                   이사 갈때마다 이거샀어요 진짜튼튼해요 이삿짐센터도 못땠습니더
6   화장대 옆에 붙였는데 1번 떨어졌어요 그리고 다시 붙이지못했어요 디자인이 예쁘고 깔...
7   드라이기가 너무 방치되어 있어서 후기들 보고 구매했어요 확실히 드라이기 정리함이 있...
8   이사오니 드라이를 놓을 곳이 없어서 구매했어요 하루 붙여놓으니 튼튼해요 헤어오일과 ...
9    튼튼하게 잘 붙어있어서 좋아요 화이트라 깔끔하고 디자인도 군더더기 없어서 마음에 들어요
10  드라이기뿐아니라 굴러다니는 빗까지 꽂을수있어 좋습니다 철제 수납장에 붙여서 사용중인...
11  무게가 생각보다 있어서 집에 있는 양면테이프 썻더니 떨어지더라구여 붙일수 있는 테이...
12    드라이어 빗을 놓고 사용중이에요 욕실이 많이 습한 편인데 접착력이 아주 좋은것 같아요
13  고정도 잘 되고 무게도 버텨줘서 잘 산 것 같습니다 벽면에 붙이기도 쉬워서 남녀노소...
14  어머니 욕실에 달아드렸어요 드라이어와 롤빗 수납이 맞춘 것처럼 사이즈 딱 맞아서 대...
15  집에 드릴도 없고 타일에 구멍 뚫기도 좀 그래서 무타공 마켓을 선택했습니다 붙이는데...
16  스티커 부착인줄 알았는데 본드더라구요 본드부터가 안따져서 당황했습니다 잘 안붙어잌 ...
17  딸이 크면서 외출준비에 함께 사용하는 공간이

In [ ]:
import pandas as pd
import torch
import gc

# 텍스트 생성 함수
def generate_summary(prompt, max_new_tokens=300, temperature=0.7):
    result = pipe(prompt, max_new_tokens=max_new_tokens, do_sample=True, temperature=temperature)
    return result[0]["generated_text"].strip()

# 프롬프트 생성 함수 + 프롬프트 길이 제한
def build_prompt(reviews, sentiment, max_prompt_length=3500):
    # 최대 길이를 초과하지 않도록 리뷰 줄 수 조절
    joined = ""
    limited_reviews = []
    for r in reviews:
        if len(joined) + len(r) + 1 > max_prompt_length:
            break
        limited_reviews.append(r)
        joined += r + "\n"

    reviews_text = "\n".join(limited_reviews)
    if sentiment == "both":
        prompt = f"""
당신은 제품 리뷰를 분석하는 전문가입니다.

아래는 고객들이 작성한 실제 리뷰입니다.
중복된 표현은 묶어서 정리하고, 핵심적인 **긍정적인 점**과 **아쉬운 점**을 각각 간결하게 3개씩 요약해 주세요.

- 각 항목은 **한 문장**으로 작성해 주세요.
- **오타 없이** 맞춤법에 맞게 작성해 주세요.
- 불분명하거나 제품과 관련 없는 표현은 제거해 주세요.

[리뷰 목록]
{reviews_text}

[요약 결과]

긍정적인 점:
1.

2.

3.

아쉬운 점:
1.

2.

3.
"""
        print(f"[사용된 리뷰 수] {len(limited_reviews)}개")
        print(f"[프롬프트 길이] {len(prompt)}자")
        return prompt

# 전체 요약 실행
def summarize_all(reviews, sentiment):
    try:
        prompt = build_prompt(reviews, sentiment)
        summary = generate_summary(prompt)
        torch.cuda.empty_cache()
        gc.collect()
        return summary
    except Exception as e:
        print(f"[ERROR] 전체 요약 실패: {e}")
        return ""

# 실행
total_reviews = df_1["review"].dropna().tolist()
final_summary = summarize_all(total_reviews, "both")

print("[전체 리뷰 요약 결과]")
print(final_summary)


[사용된 리뷰 수] 43개
[프롬프트 길이] 3757자
[전체 리뷰 요약 결과]
```

**긍정적인 점:**
1. 튼튼하게 잘 붙어있어서 좋아요.
2. 다양한 물건 수납까지 할 수 있어서 좋네요.
3. 깔끔하고 딱 필요한만큼 모던하니 좋아요.

**아쉬운 점:**
1. 고무패킹이 잘빠지네요.
2. 빗 꽂아두는 곳 밑 면이 뚫려 있어서 종종 빗이 떨어지는 게 좀 아쉬워요.
3. 드라이기 닿는 부분에 투명 고무 부분이 있는데 잘 고정되지 않고 자꾸 빠져서 넣고 뺄때 불편합니다.


### 수건

In [ ]:
import pandas as pd

# 파일 경로
file_path = '/content/drive/MyDrive/KUBIG/25_1_conference/crawling_result/345755_전체리뷰_전처리결과.txt'

# 줄 단위로 읽기 (공백 포함)
with open(file_path, 'r', encoding='utf-8') as f:
    lines = f.readlines()
# [전체 리뷰 - 50개] 같은 0번째 줄 제거
if lines and "[전체 리뷰" in lines[0]:
    lines = lines[1:]

# 줄바꿈 및 공백 제거
cleaned_reviews = [line.strip() for line in lines if line.strip()]
df_1 = pd.DataFrame(cleaned_reviews, columns=['review'])

# 결과 출력
print(df_1)


                                               review
0   딸아이 기숙사가는데 보내려고 주문했어요 제법 도톰하고 폭신해서 좋구요 세탁 1번했는...
1   이사하면서 수건 바꿨는데 보기만 해도 기분이 좋네요 관리하기 힘들까봐 진한 색으로 ...
2   이사하면서 수건 멀쩡한거 두장 빼고 다 새 수건으로 갈았어요 오집에서 어제 주문했는...
3   색감도 딱 원하던거고 부들부들 잘산거같아요 가격대비 최고예요 밝은색 사고싶었지만 금...
4   요즘에 선물할 일 있으면 꼭 수건으로 하게 되네요 예전에는 수건 선물이 종종 들어왓...
5   자취를 시작하면서 수건을 새로 샀었는데 다른 수건들에 비해 두께가 도톰해서 물기도 ...
6   수건이 부드럽고 두툼해서 물기를 잘 흡수하긴 하는데 다른 리뷰들처럼 금방 마르진 않...
7   신혼집 장만하면서 수건도 새로 샀습니다 고중량 호텔수건 사니까 있어보이고 좋네요 컬...
8   노란색 흰색 20장씩 주문이용 수건교체시기가 한참 지났지만 귀찮아서 그냥 썼는데 이...
9   처음 세탁 할 때 세제 넣지 말고 알려 주신 방법 그대로 하는 유지가 잘 되는 것 ...
10  아래가 200 짜리에요 다좋은데 폭신 부들 저희집엔 화장실 장이 작아서 200 짜리...
11  저희 집 입주때 사서 잘 쓰다가 엄마집 이사가셔서 수건 바꿔드렸어요 제 기준엔 20...
12  딸아이가 처음으로 원룸생활 하는데 필요해서 구매했는데 원단도톰하고 털빠짐이 없어서 ...
13  이사하면서 새수건으로 구매했습니대 컬러감도 좋고 배송도 빠르고 무엇보다 촉감이 너무...
14  저희집에 쓰고있는데 좋아서 오빠 이사하는데 주려고 세탁건조 돌렸습니다 피스타치오 색...
15  기존에 회색쓰고 있었는데 몇개 엄마 드렸더니 당장 다른색 여러개 사달라하셔서 또 주...
16  포장꼼꼼하고 배송빨라요 수세미도 사은품으로 주셨어요 타월색상 고민했는데 파스타치오 ...
17  첨에 회색 사고 넘 두껍고 좋아서 다른 색상

In [ ]:
import pandas as pd
import torch
import gc

# 텍스트 생성 함수
def generate_summary(prompt, max_new_tokens=300, temperature=0.7):
    result = pipe(prompt, max_new_tokens=max_new_tokens, do_sample=True, temperature=temperature)
    return result[0]["generated_text"].strip()

# 프롬프트 생성 함수 + 프롬프트 길이 제한
def build_prompt(reviews, sentiment, max_prompt_length=3500):
    # 최대 길이를 초과하지 않도록 리뷰 줄 수 조절
    joined = ""
    limited_reviews = []
    for r in reviews:
        if len(joined) + len(r) + 1 > max_prompt_length:
            break
        limited_reviews.append(r)
        joined += r + "\n"

    reviews_text = "\n".join(limited_reviews)
    if sentiment == "both":
        prompt = f"""
당신은 제품 리뷰를 분석하는 전문가입니다.

아래는 고객들이 작성한 실제 리뷰입니다.
중복된 표현은 묶어서 정리하고, 핵심적인 **긍정적인 점**과 **아쉬운 점**을 각각 간결하게 3개씩 요약해 주세요.

- 각 항목은 **한 문장**으로 작성해 주세요.
- **오타 없이** 맞춤법에 맞게 작성해 주세요.
- 불분명하거나 제품과 관련 없는 표현은 제거해 주세요.

[리뷰 목록]
{reviews_text}

[요약 결과]

긍정적인 점:
1.

2.

3.

아쉬운 점:
1.

2.

3.
"""
        print(f"[사용된 리뷰 수] {len(limited_reviews)}개")
        print(f"[프롬프트 길이] {len(prompt)}자")
        return prompt

# 전체 요약 실행
def summarize_all(reviews, sentiment):
    try:
        prompt = build_prompt(reviews, sentiment)
        summary = generate_summary(prompt)
        torch.cuda.empty_cache()
        gc.collect()
        return summary
    except Exception as e:
        print(f"[ERROR] 전체 요약 실패: {e}")
        return ""


# 실행
total_reviews = df_1["review"].dropna().tolist()
final_summary = summarize_all(total_reviews, "both")

print("[전체 리뷰 요약 결과]")
print(final_summary)


[사용된 리뷰 수] 24개
[프롬프트 길이] 3745자
[전체 리뷰 요약 결과]
```

**긍정적인 점:**

1. 두터른 두께와 흡수력으로 물기를 잘 흡수합니다.
2. 디자인이 호텔에서 봤던 그런 디자인이라 수건장에 넣어두니 수건 하나만으로도 뭔가 삶의질이 확 올라간 느낌이 드네요.
3. 편안한 촉감과 무겁게 흡수되는 특징으로 매일 사용하기 좋습니다.

**아쉬운 점:**

1. 세탁 후 변형이나 색 바램이 많이 발생합니다.
2. 건조기가 없어서 세탁 후 먼지가 많이 나오는 경우가 많습니다.
3. 빨래에 닿혀 빨레하는 모든 빨랫 감이 파란 먼지 투성이에요.


## llama

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# ──────────────────────────────
# 1. 모델 이름 지정
# ──────────────────────────────
model_name = "meta-llama/Llama-2-7b-chat-hf"  # 또는 Llama-2-13b, Hugging Face에 토큰 필요

# ──────────────────────────────
# 2. 4-bit 양자화 설정
# ──────────────────────────────
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# ──────────────────────────────
# 3. 모델 및 토크나이저 로드
# ──────────────────────────────
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

# ──────────────────────────────
# 4. LoRA 구성 및 적용
# ──────────────────────────────
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
model.eval()
model.half()  # optional: ensures fp16 everywhere

# ──────────────────────────────
# 5. 텍스트 생성 파이프라인 구성
# ──────────────────────────────
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
    return_full_text=False
)



#streamlit

In [ ]:
!pip install -q transformers accelerate bitsandbytes peft
!pip install -q selenium
!apt-get update -q
!apt install chromium-chromedriver -y
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 76.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4

In [ ]:
import re
import time
import traceback
import pandas as pd
import torch
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# --- 크롤러 클래스 ---
class TodayHouseSimpleCrawler:
    def __init__(self, url):
        options = Options()
        options.add_argument("--headless")
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("lang=ko_KR")
        options.add_argument("user-agent=Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 Chrome/88.0.4324.96 Safari/537.36")
        self.driver = webdriver.Chrome(options=options)
        self.wait = WebDriverWait(self.driver, 5)
        self.url = url
        self.seen_reviews = set()

    def collect_reviews(self, max_reviews=50):
        self.driver.get(self.url)
        try:
            review_button = self.wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'a[href="#production-selling-review"]')))
            self.driver.execute_script("arguments[0].click();", review_button)
            self.wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "article.production-review-item")))
        except Exception as e:
            print(f"[ERROR] 리뷰 요소 로딩 실패: {e}")
            self.driver.quit()
            return []

        collected = []
        while len(collected) < max_reviews:
            try:
                self.wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "article.production-review-item")))
                tags = self.driver.find_elements(By.CSS_SELECTOR, "p.production-review-item__description")
                for tag in tags:
                    if len(collected) >= max_reviews:
                        break
                    text = tag.text.strip()
                    if text and text not in self.seen_reviews:
                        collected.append(text)
                        self.seen_reviews.add(text)
                try:
                    next_btn = self.driver.find_element(By.CSS_SELECTOR, "button._3b4ci:last-child")
                    self.driver.execute_script("arguments[0].click();", next_btn)
                    time.sleep(1)
                except:
                    break
            except Exception as e:
                print(f"[ERROR] 리뷰 수집 중 오류: {e}")
                break

        self.driver.quit()
        return collected

# --- 리뷰 크롤링 ---
def crawl_todayhouse_50_reviews(url):
    if not re.search(r"/productions/(\d+)", url):
        raise ValueError("올바른 오늘의집 URL이 아닙니다.")
    crawler = TodayHouseSimpleCrawler(url)
    reviews = crawler.collect_reviews(50)
    return pd.DataFrame([r.replace("\n", " ").strip() for r in reviews], columns=["Review"])


In [ ]:
# --- 모델 로딩 ---
def load_lora_model():
    model_name = "google/gemma-2b-it"
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16
    )
    lora_config = LoraConfig(
        r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM"
    )
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto",
        torch_dtype=torch.float16
    )
    model = prepare_model_for_kbit_training(model)
    model = get_peft_model(model, lora_config)
    model.half()
    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, torch_dtype=torch.float16, device_map="auto", return_full_text=False)
    return pipe

# --- 요약 생성 ---
def build_prompt(reviews, sentiment, max_len=3500):
    joined = ""
    limited = []
    for r in reviews:
        if len(joined) + len(r) + 1 > max_len: break
        joined += r + "\n"
        limited.append(r)
    prompt = f"""
당신은 제품 리뷰를 분석하는 전문가입니다. 다음은 리뷰 모음입니다.

리뷰를 분석하여 다음 지침에 따라 핵심 내용을 요약하세요:
1. 비슷한 의견은 하나로 묶고 중복 없이 정리하세요.
2. 긍정적인 점과 아쉬운 점으로 나누어 요약하세요.
3. 각 항목은 간결한 한 문장으로 작성하세요.
4. 제품과 무관한 내용은 제외하세요.

[리뷰 목록]
{joined}

[요약 결과]
"""
    return prompt

def summarize_chunks(pipe, chunks, sentiment="both"):
    summaries = []
    for chunk in chunks:
        try:
            prompt = build_prompt(chunk, sentiment)
            summary = pipe(prompt, max_new_tokens=300, do_sample=True, temperature=0.7)[0]["generated_text"]
            summaries.append(summary.strip())
        except Exception as e:
            print(f"[요약 실패]: {e}")
    return "\n\n".join(summaries)

def split_reviews(reviews, chunk_size=50):
    return [reviews[i:i+chunk_size] for i in range(0, len(reviews), chunk_size)]


In [ ]:
# --- 실행 ---
def run():
    url = input("오늘의집 제품 URL을 입력하세요: ").strip()
    df = crawl_todayhouse_50_reviews(url)
    print("수집된 리뷰:", df.head())
    reviews = df["Review"].tolist()
    chunks = split_reviews(reviews)
    pipe = load_lora_model()
    summary = summarize_chunks(pipe, chunks)
    print("\n===== 요약 결과 =====\n")
    print(summary)

run()

오늘의집 제품 URL을 입력하세요: https://ohou.se/productions/1972953/selling


수집된 리뷰:                                               Review
0  장마기간을 준비하면서 구입했어요 바빠서 아직 뜯어보진 못했지만 후기가 좋아 기대하고...
1  높이가 낮은 대신 와이드형이라 옷장이나 신발장 등 가구 밑에 두기에 좋음 어디 걸리...
2  오래된 아파트라 리모델링을 했는데도 붙박이장에 습기가 많이 차서 항상 쟁여놓고 쓰는...
3  사각이라 옷장안에 딱떨어지게 넣기 좋네요 곧 장마시작된다고 하고 비오면 습도도 많이...
4  장마전 미리 구매하는 필수템 올해도 장만해서 집안.구석 구석 박아놨어요. 금새 물 ...


tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Device set to use cuda:0



===== 요약 결과 =====

- 비슷한 의견: 옷장안의 방향과 습도 관리
- 긍정적인 점: 장마의 길이, 편의성, 용량, 옷장 안의 방향
- 긍정적인 점: 옷장 안의 습도 관리
- 긍정적인 점: 옷의 양과 신발의 양

**핵심 내용:**

* 옷장 안의 방향과 습도 관리가 중요합니다.
* 장마의 길이, 편의성, 용량, 옷장 안의 방향은 긍정적인 평가를 받습니다.
* 옷장 안의 습도 관리는 긍정적인 평가를 받습니다.


In [ ]:
# 코랩에서 크롤링 + 요약 (Gemma), 결과는 Streamlit UI에서 보기 위해 결과 파일로 저장

!pip install -q transformers accelerate bitsandbytes peft selenium
!apt-get update -q
!apt install chromium-chromedriver -y
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

import re
import time
import pandas as pd
import traceback
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# --- 크롤러 ---
class TodayHouseSimpleCrawler:
    def __init__(self, url):
        options = Options()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('lang=ko_KR')
        options.add_argument("user-agent=Mozilla/5.0 (X11; Linux x86_64)")
        self.driver = webdriver.Chrome(options=options)
        self.wait = WebDriverWait(self.driver, 5)
        self.url = url
        self.seen_reviews = set()

    def collect_reviews(self, max_reviews=50):
        self.driver.get(self.url)
        try:
            review_btn = self.wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'a[href="#production-selling-review"]')))
            self.driver.execute_script("arguments[0].click();", review_btn)
            self.wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "article.production-review-item")))
        except:
            self.driver.quit()
            return []

        reviews = []
        while len(reviews) < max_reviews:
            tags = self.driver.find_elements(By.CSS_SELECTOR, "p.production-review-item__description")
            for tag in tags:
                if len(reviews) >= max_reviews:
                    break
                text = tag.text.strip()
                if text and text not in self.seen_reviews:
                    reviews.append(text)
                    self.seen_reviews.add(text)
            try:
                next_btn = self.driver.find_element(By.CSS_SELECTOR, "button._3b4ci:last-child")
                self.driver.execute_script("arguments[0].click();", next_btn)
                time.sleep(1)
            except:
                break

        self.driver.quit()
        return reviews

# --- 요약 ---
def load_lora_model():
    model_name = "google/gemma-2b-it"
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16
    )
    lora_config = LoraConfig(
        r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM"
    )
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto",
        torch_dtype=torch.float16
    )
    model = prepare_model_for_kbit_training(model)
    model = get_peft_model(model, lora_config)
    model.half()
    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, return_full_text=False, torch_dtype=torch.float16, device_map="auto")
    return pipe

def build_prompt(reviews):
    joined = "\n".join(reviews[:50])
    return f"""
당신은 제품 리뷰를 분석하는 전문가입니다. 다음은 리뷰 모음입니다.

리뷰를 분석하여 다음 지침에 따라 핵심 내용을 요약하세요:
1. 비슷한 의견은 하나로 묶고 중복 없이 정리하세요.
2. 긍정적인 점과 아쉬운 점으로 나누어 요약하세요.
3. 각 항목은 간결한 한 문장으로 작성하세요.
4. 제품과 무관한 내용은 제외하세요.

[리뷰 목록]
{joined}

[요약 결과]
"""

# --- 전체 실행 ---
def run():
    url = input("오늘의집 제품 URL 입력: ").strip()
    crawler = TodayHouseSimpleCrawler(url)
    reviews = crawler.collect_reviews(50)
    print(f"총 {len(reviews)}개 리뷰 수집 완료")

    pipe = load_lora_model()
    prompt = build_prompt(reviews)
    summary = pipe(prompt, max_new_tokens=300, do_sample=True, temperature=0.7)[0]['generated_text']

    with open("summary.txt", "w", encoding="utf-8") as f:
        f.write(summary.strip())
    print("summary.txt 파일로 저장 완료")

run()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 129 kB in 1s (128 kB/s)
Reading package lists...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Buil

총 5개 리뷰 수집 완료


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


summary.txt 파일로 저장 완료


# final

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!huggingface-cli login

In [ ]:
# --- 설치 및 셋업 ---
!apt-get update
!apt-get install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium chromedriver-autoinstaller

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [ ]:
import sys, os, re, json, time
import torch
import pandas as pd
from google.colab import drive
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import bitsandbytes as bnb
from transformers import (
    AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# --- Google Drive 마운트 ---
drive.mount('/content/drive')
save_path = '/content/drive/MyDrive/ohou_summary/summary_result.json'
os.makedirs(os.path.dirname(save_path), exist_ok=True)

# --- Chrome 설정 ---
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("lang=ko_KR")
chrome_options.add_argument("user-agent=Mozilla/5.0")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# --- 오늘의집 크롤러 ---
class TodayHousePageNumberCrawler:
    def __init__(self, url):
        self.url = url
        self.driver = webdriver.Chrome(options=chrome_options)
        self.wait = WebDriverWait(self.driver, 10)
        self.seen_reviews = set()

    def collect_reviews(self, max_reviews=50):
        self.driver.get(self.url)
        collected_reviews = []
        page = 1

        while len(collected_reviews) < max_reviews:
            try:
                self.wait.until(EC.presence_of_all_elements_located(
                    (By.CSS_SELECTOR, "p.production-review-item__description")))
                tags = self.driver.find_elements(By.CSS_SELECTOR, "p.production-review-item__description")

                for tag in tags:
                    review_text = tag.text.strip()
                    if review_text and review_text not in self.seen_reviews:
                        collected_reviews.append(review_text)
                        self.seen_reviews.add(review_text)
                        if len(collected_reviews) >= max_reviews:
                            break

                page += 1
                next_page_btn = self.driver.find_element(
                    By.XPATH,
                    f"//button[@class='_3b4ci' and normalize-space(text())='{page}']"
                )
                self.driver.execute_script("arguments[0].click();", next_page_btn)
                self.wait.until(EC.staleness_of(tags[0]))

            except Exception as e:
                print(f"[INFO] {page}페이지 없음 또는 이동 실패: {e}")
                break

        self.driver.quit()
        return collected_reviews


In [ ]:
# --- Gemma 요약 모델 준비 ---
model_name = "google/gemma-7b-it"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16,
    quantization_config=bnb_config
)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
model.eval()


tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/2.11G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GemmaForCausalLM(
      (model): GemmaModel(
        (embed_tokens): Embedding(256000, 3072, padding_idx=0)
        (layers): ModuleList(
          (0-27): 28 x GemmaDecoderLayer(
            (self_attn): GemmaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              

In [ ]:
def build_prompt_for_gemma(reviews):
    joined = "\n".join(f"{i+1}. {r}" for i, r in enumerate(reviews))

    prompt = (
        "다음은 한국어 상품 리뷰입니다.\n\n"
        "리뷰들을 바탕으로 다음과 같이 요약해 주세요:\n"
        "1. 전체적인 한줄 요약\n"
        "2. 장점들 (중복 없이)\n"
        "3. 단점들 (중복 없이)\n\n"
        f"{joined}\n\n"
        "한줄 요약:"
    )
    return prompt

def summarize_with_gemma(reviews, tokenizer, model):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    prompt = build_prompt_for_gemma(reviews)
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.7,
        top_p=0.95,
        eos_token_id=tokenizer.eos_token_id
    )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = decoded.split("한줄 요약:")[-1].strip()

    lines = response.split("\n")
    one_line = ""
    pros, cons = [], []
    mode = "summary"

    for line in lines:
        line = line.strip()
        if not line:
            continue
        if "장점" in line:
            mode = "pros"
            continue
        if "단점" in line:
            mode = "cons"
            continue
        if mode == "summary" and not one_line:
            one_line = line
        elif mode == "pros":
            pros.append(line.lstrip("-•1234567890. "))
        elif mode == "cons":
            cons.append(line.lstrip("-•1234567890. "))

    return {
        "one_line": one_line.strip(),
        "pros": pros,
        "cons": cons
    }


In [ ]:
# --- 실행부 ---
url = input("오늘의집 제품 URL을 입력하세요: ").strip()
if not re.search(r"/productions/\d+", url):
    print("올바른 오늘의집 제품 링크를 입력하세요.")
else:
    crawler = TodayHousePageNumberCrawler(url)
    reviews = crawler.collect_reviews(max_reviews=50)
    cleaned_reviews = [r.replace("\n", " ").strip() for r in reviews]

    print()
    print(f"리뷰 {len(cleaned_reviews)}개 수집 완료. Gemma로 요약 중...")
    print()

    # 요약 수행 + 전체 출력 결과 포함
    summary = summarize_with_gemma(cleaned_reviews, tokenizer, model)

    # 전체 출력 결과
    print("Gemma 전체 출력:")
    print("=" * 60)
    print(summary["full_text"])
    print("=" * 60)
    print()

    # 정리된 요약 결과 출력
    print("한줄 요약:")
    print(summary["one_line"])
    print()

    if summary["pros"]:
        print("장점:")
        for point in summary["pros"]:
            print(f"- {point}")
        print()
    else:
        print("장점: 없음")
        print()

    if summary["cons"]:
        print("단점:")
        for point in summary["cons"]:
            print(f"- {point}")
        print()
    else:
        print("단점: 없음")
        print()

    # JSON 저장
    result = {
        "one_line": summary["one_line"],
        "pros": summary["pros"],
        "cons": summary["cons"],
        "full_text": decoded  # 여기를 꼭 포함
    }

    with open(save_path, "w", encoding="utf-8") as f:
        json.dump(result, f, ensure_ascii=False, indent=2)

    print(f"요약 결과가 저장되었습니다: {save_path}")


오늘의집 제품 URL을 입력하세요: https://ohou.se/productions/1972953/selling

리뷰 50개 수집 완료. Gemma로 요약 중...

Gemma 전체 출력:


KeyError: 'full_text'